In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_formatting import *
from collections import Counter
import numpy as np
import time

import pandas as pd
import geopandas as gpd
import datetime as dt

import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
from matplotlib import ticker
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.ticker import MaxNLocator

import itertools as it

plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor']='white'
plt.rcParams['savefig.facecolor']='white'
plt.rc('axes', axisbelow=True)
%matplotlib inline

In [2]:
today = time.strftime("%Y-%m-%d")

month_dict = {'01':'Jan','02':'Feb','03':'Mar','04':'Apr','05':'May','06':'Jun',
              '07':'Jul','08':'Aug','09':'Sep','10':'Oct','11':'Nov','12':'Dec'}

weekday_mapping = {0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'}

alldates = pd.date_range(start='2020-01-06', end='2021-12-31').tolist()
datelist = [str(i.date()) for i in alldates]
weekdays = [weekday_mapping[i.date().weekday()] for i in alldates]
weeklist = [i//7 for i in range(len(alldates))]
DATE_DF = pd.DataFrame({'date':datelist,'day':weekdays,'week':weeklist})
DATERANGE = DATE_DF['date'].tolist()

date_dict = {DATERANGE[i]:i for i in range(len(DATERANGE))}

In [21]:
# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

creds = ServiceAccountCredentials.from_json_keyfile_name('../client_secret.json', scope)
client = gspread.authorize(creds)

fld = client.open('COVID-data_Florida')
names_df = pd.DataFrame(fld.worksheet('school_names').get_all_records(default_blank=np.nan))

In [4]:
fla_data1 = client.open("COVID-data_Florida_schools1")
fla_data2 = client.open("COVID-data_Florida_schools2")
fla_data3 = client.open("COVID-data_Florida_schools3")
sch_df = pd.DataFrame(fla_data1.worksheet('ref').get_all_records(default_blank=np.nan))
sch_df = sch_df[['dis_num', 'school_id', 'school', 'county', 'city', 'active']].copy()

sch_df.loc[sch_df['school'].str.contains(','),'school'] = sch_df.loc[sch_df['school'].str.contains(','),'school'].str.replace(',','')
sch_df['school'] = sch_df['school'].str.replace('.','. ')
sch_df['school'] = sch_df['school'].str.replace('.','')
sch_df['school'] = sch_df['school'].str.replace('  ',' ')

sch_df.loc[sch_df['school'].str.contains('MIDDLE SCHOOL'),
           'school'] = sch_df.loc[sch_df['school'].str.contains('MIDDLE SCHOOL'),
                                  'school'].str.replace('MIDDLE SCHOOL','MIDDLE')

sch_df.loc[sch_df['school'].str.contains('ELEMENTARY SCHOOL'),
       'school'] = sch_df.loc[sch_df['school'].str.contains('ELEMENTARY SCHOOL'),
                          'school'].str.replace('ELEMENTARY SCHOOL','ELEMENTARY')

sch_df.loc[sch_df['school'].str.contains('HIGH SCHOOL'),
           'school'] = sch_df.loc[sch_df['school'].str.contains('HIGH SCHOOL'),
                                  'school'].str.replace('HIGH SCHOOL','HIGH')

sch_df.loc[sch_df['school'].str.contains('SENIOR HIGH'),
           'school'] = sch_df.loc[sch_df['school'].str.contains('SENIOR HIGH'),
                                  'school'].str.replace('SENIOR HIGH','HIGH')

In [5]:
worksheets1 = [i.title for i in fla_data1.worksheets() if i.title != 'ref']
worksheets2 = [i.title for i in fla_data2.worksheets() if i.title != 'ref']
worksheets3 = [i.title for i in fla_data3.worksheets() if i.title != 'ref']

In [6]:
fl_dfs = []
for i,ix in enumerate(worksheets1):
    print(i,ix)
    dfi = pd.DataFrame(fla_data1.worksheet(ix).get_all_records(default_blank=np.nan))
    fl_dfs.append(dfi)

0 2020-09-26
1 2020-10-03
2 2020-10-17
3 2020-10-24
4 2020-10-31
5 2020-11-07
6 2020-11-14
7 2020-11-21
8 2020-11-28
9 2020-12-05
10 2020-12-12
11 2020-12-19
12 2020-12-26
13 2021-01-02
14 2021-01-09
15 2021-01-16
16 2021-01-23
17 2021-01-30


In [7]:
for i,ix in enumerate(worksheets2):
    print(i,ix)
    dfi = pd.DataFrame(fla_data2.worksheet(ix).get_all_records(default_blank=np.nan))
    fl_dfs.append(dfi)

0 2021-02-06
1 2021-02-13
2 2021-02-20
3 2021-02-27
4 2021-03-06


In [9]:
# for i,ix in enumerate(worksheets3):
#     print(i,ix)
#     dfi = pd.DataFrame(fla_data3.worksheet(ix).get_all_records(default_blank=np.nan))
#     fl_dfs.append(dfi)

for i,ix in enumerate(worksheets3[3:]):
    print(i,ix)
    dfi = pd.DataFrame(fla_data3.worksheet(ix).get_all_records(default_blank=np.nan))
    fl_dfs.append(dfi)

0 2021-04-03
1 2021-04-10
2 2021-04-17
3 2021-04-24
4 2021-05-01
5 2021-05-08
6 2021-05-15
7 2021-05-22


In [22]:
fl_df = pd.concat(fl_dfs).reset_index().iloc[:,1:]
fl_df = fl_df[['date', 'start_date', 'school', 'county', 'new_cases',
       'new_cases_students', 'new_cases_teachers', 'new_cases_staff',
       'new_cases_unknown', 'symptoms_yes', 'symptoms_no', 'symptoms_idk',
       'cumulative_cases', 'cumulative_cases_students',
       'cumulative_cases_teachers', 'cumulative_cases_staff',
       'cumulative_cases_unknown', 'type']].copy()
fl_df['county'] = fl_df['county'].astype(str)
fl_df['school'] = fl_df['school'].astype(str)
fl_df['county'] = fl_df['county'].str.upper()

fl_df.loc[fl_df['county']=='DADE','county'] = 'MIAMI-DADE'
fl_df['school'] = fl_df['school'].str.upper()
fl_df.loc[(fl_df['school'].str.contains("\("))&\
          (~fl_df['school'].str.contains("\)")),
          'school'] = fl_df.loc[(fl_df['school'].str.contains("\("))&\
                                (~fl_df['school'].str.contains("\)")),
                                'school'].str[:] + ")"
fl_df['school'] = fl_df['school'].str.lstrip(' ')
fl_df['school'] = fl_df['school'].str.rstrip(' ')
fl_df['school'] = fl_df['school'].str.replace('  ',' ')
fl_df['school'] = fl_df['school'].str.replace("S\(O","S (O").str.replace("I\(H","I (H")
fl_df.loc[(fl_df['date']<'2020-11-28'),'school'] = fl_df.loc[(fl_df['date']<'2020-11-28'),'school'].str.rstrip('\)')
fl_df.loc[(fl_df['date']<'2020-11-28')&\
          (fl_df['school'].str.contains("\("))&\
          (~fl_df['school'].str.contains("\)")),
          'school'] = fl_df.loc[(fl_df['date']<'2020-11-28')&\
                      (fl_df['school'].str.contains("\("))&\
                      (~fl_df['school'].str.contains("\)")),
                      'school'].str.split(" \(").str[0]

In [23]:
curr_sch = fl_df[['school','county']].drop_duplicates()

In [24]:
pd.concat([names_df,curr_sch.loc[~curr_sch['school'].isin(names_df['school'])]]).to_csv('../data/fl_schools_names_apr17.tsv',sep='\t',index=False)

In [25]:
fl_df = fl_df.merge(names_df, how='left', on=['school','county'])

In [26]:
# fl_dfx = fl_df.copy()

In [27]:
fl_df['school'] = fl_df['school_new']
fl_df['county'] = fl_df['county_new']
fl_df = fl_df[['date', 'start_date', 'school', 'county', 'new_cases',
       'new_cases_students', 'new_cases_teachers', 'new_cases_staff',
       'new_cases_unknown', 'symptoms_yes', 'symptoms_no', 'symptoms_idk',
       'cumulative_cases', 'cumulative_cases_students',
       'cumulative_cases_teachers', 'cumulative_cases_staff',
       'cumulative_cases_unknown', 'type']].copy()
fl_df = fl_df.loc[~fl_df['school'].isna()].copy()

In [28]:
fl_df.loc[(fl_df['school'].str.contains('ELEMENTARY'))&(fl_df['type']=='university'),'type'] = 'k12'
fl_df.loc[(fl_df['school'].str.contains('MIDDLE'))&(fl_df['type']=='university'),'type'] = 'k12'
fl_df.loc[(fl_df['school'].str.contains('CHARTER'))&(fl_df['type']=='university'),'type'] = 'k12'
fl_df.loc[(fl_df['school'].str.contains('HIGH'))&(fl_df['type']=='university'),'type'] = 'k12'

fl_df.loc[(fl_df['school'].str.contains('COLLEGE'))&\
          ~(fl_df['school'].str.contains('HIGH'))&\
          ~(fl_df['school'].str.contains('CHARTER'))&\
          ~(fl_df['school'].str.contains('ELEM'))&\
          ~(fl_df['school'].str.contains('ACADEMY'))&\
          ~(fl_df['school'].str.contains('SCHOOL'))&\
          ~(fl_df['school'].str.contains('CENTER'))&\
          ~(fl_df['school'].str.contains('MAST@'))&\
          ~(fl_df['school'].str.contains('PREP'))&\
          ~(fl_df['school'].str.contains('COLLEGIATE'))&\
          (fl_df['type']=='k12'),'type'] = 'university'

fl_df.loc[(fl_df['school'].str.contains('UNIV'))&\
          ~(fl_df['school'].str.contains('HIGH'))&\
          ~(fl_df['school'].str.contains('CHARTER'))&\
          ~(fl_df['school'].str.contains('ELEM'))&\
          ~(fl_df['school'].str.contains('ACADEMY'))&\
          ~(fl_df['school'].str.contains('SCHOOL'))&\
          ~(fl_df['school'].str.contains('CENTER'))&\
          ~(fl_df['school'].str.contains('MAST@'))&\
          ~(fl_df['school'].str.contains('PREP'))&\
          ~(fl_df['school'].str.contains('COLLEGIATE'))&\
          (fl_df['type']=='k12'),'type'] = 'university'
# ][['school','county','type']].drop_duplicates()#

fl_df.loc[fl_df['school']=='ASBURY CHRISTIAN SCHOOL','type'] = 'k12'
fl_df.loc[fl_df['school']=='DAVID POSNACK SCHOOL IN DAVIE','type'] = 'k12'
fl_df.loc[fl_df['school']=='DEANE BOZEMAN SCHOOL','type'] = 'k12'
fl_df.loc[fl_df['school']=='FIELDS OF DREAMS SPORTS ACADEMY','type'] = 'k12'
fl_df.loc[fl_df['school'].str.contains('FIRST ACADEMY'),'type'] = 'k12'
fl_df.loc[fl_df['school']=='IMG ACADEMY','type'] = 'k12'
fl_df.loc[fl_df['school']=='KATZ HILLEL DAY SCHOOL','type'] = 'k12'
fl_df.loc[fl_df['school']=='L3 HARRIS FLIGHT SCHOOL','type'] = 'university'
fl_df.loc[fl_df['school']=='LEON COUNTY PUBLIC SCHOOLS','type'] = 'k12'
fl_df.loc[fl_df['school']=='LINCOLN-MARTI COMMUNITY AGENCY 13','type'] = 'k12'
fl_df.loc[fl_df['school']=='MAGNOLIA SCHOOL','type'] = 'k12'
fl_df.loc[fl_df['school']=='MIAMI COUNTRY DAY SCHOOL','type'] = 'k12'
fl_df.loc[fl_df['school']=='MIAMI DADE PUBLIC SCHOOL','type'] = 'k12'
fl_df.loc[fl_df['school'].str.contains('MIAMI LAKES EDUCATIONAL CENTER AND'),'type'] = 'k12'
fl_df.loc[fl_df['school'].str.contains('MIAMI LAKES K-8 CENTER'),'type'] = 'k12'
fl_df.loc[fl_df['school']=='PASCO COUNTY SCHOOL DISTRICT','type'] = 'k12'
fl_df.loc[fl_df['school']=='RCMA WIMAUMA COMMUNITY ACADEMY','type'] = 'k12'
fl_df.loc[fl_df['school']=='SCHOOLHOUSE ACADEMY','type'] = 'k12'
fl_df.loc[fl_df['school']=='SPANISH RIVER CHRISTIAN','type'] = 'k12'
fl_df.loc[fl_df['school']=='STANTON COLLEGE PREPARATORY','type'] = 'k12'
fl_df.loc[fl_df['school']=='SUMMIT-QUESTA MONTESSORI SCHOOL','type'] = 'k12'
fl_df.loc[fl_df['school']=='THE CARRIE BRAZER CENTER FOR AUTISM INC.','type'] = 'k12'
fl_df.loc[fl_df['school']=='HOLLYWOOD INSTITUTE OF BEAUTY CAREERS','type'] = 'university'


In [29]:
tmp = fl_df[['school','county','type']].drop_duplicates()#
tmp['x'] = 1
gmp = tmp.groupby(['school','county']).agg({'x':'sum'}).reset_index()

In [30]:
gbcol = ['school','county','date','start_date','type']
agcol = ['new_cases', 'new_cases_students', 'new_cases_teachers', 'new_cases_staff',
         'new_cases_unknown', 'symptoms_yes', 'symptoms_no', 'symptoms_idk',
         'cumulative_cases', 'cumulative_cases_students', 'cumulative_cases_teachers',
         'cumulative_cases_staff', 'cumulative_cases_unknown']

In [31]:
fl_schools = fl_df.groupby(gbcol).agg({i:np.nansum for i in agcol}).reset_index()

In [32]:
fl_schools['school'] = fl_schools['school'].str.title()
fl_schools['school'] = fl_schools['school'].str.replace("'S","'s")
fl_schools['county'] = fl_schools['county'].str.title()

fl_schools['cumulative_cases'] = fl_schools.groupby(['school','county','type'])['new_cases'].transform(lambda x: x.cumsum()) 
fl_schools['cumulative_cases_students'] = fl_schools.groupby(['school','county','type'])['new_cases_students'].transform(lambda x: x.cumsum()) 
fl_schools['cumulative_cases_teachers'] = fl_schools.groupby(['school','county','type'])['new_cases_teachers'].transform(lambda x: x.cumsum()) 
fl_schools['cumulative_cases_staff'] = fl_schools.groupby(['school','county','type'])['new_cases_staff'].transform(lambda x: x.cumsum()) 
fl_schools['cumulative_cases_unknown'] = fl_schools.groupby(['school','county','type'])['new_cases_unknown'].transform(lambda x: x.cumsum()) 
flg = fl_schools.loc[fl_schools['type']=='k12'].groupby('date').agg({i:'sum' for i in agcol}).reset_index()

In [33]:
# fl_schools.to_csv('../data/fl_schools_apr24.csv',index=False)

In [34]:
fl_schools.to_csv('../data/fl_schools_may15.tsv',sep='\t',index=False)

In [51]:
fl_df[['school','county','type']].drop_duplicates().sort_values('school').to_csv('../data/tmp.csv',index=False)

In [9]:
# fl_df.sort_values(['county','school','date']).reset_index().iloc[:,1:].to_csv('../data/fl_schools.csv',index=False)

In [10]:
# fl_df[['school','county']].drop_duplicates().to_csv('../data/fl_schools.csv',index=False)

In [24]:
# sch_df.to_csv('../data/fl_schools_ref.csv',index=False)
tmp_fl = pd.read_csv('../data/fl_schools_names.csv')

In [25]:
fl_dfx = fl_df.merge(tmp_fl, how='left', on=['school','county'])

In [28]:
fl_dfx.loc[fl_dfx.school_new.isna()]

,date,start_date,school,county,new_cases,new_cases_students,new_cases_teachers,new_cases_staff,new_cases_unknown,symptoms_yes,symptoms_no,symptoms_idk,cumulative_cases,cumulative_cases_students,cumulative_cases_teachers,cumulative_cases_staff,cumulative_cases_unknown,type,school_new,county_new
1269,2020-09-26,2020-09-20,UNKNOWN,UNKNOWN,8.0,3.0,3.0,0.0,2.0,42,7.0,5.0,54,29,6.0,0.0,19.0,k12,NaN,NaN
1417,2020-09-26,2020-09-20,BROWN ACADEMY,UNKNOWN,NaN,NaN,NaN,NaN,NaN,1,0.0,0.0,1,0,0.0,0.0,1.0,university,NaN,NaN
1424,2020-09-26,2020-09-20,GWINNETT INSTITUE,UNKNOWN,1.0,1.0,0.0,0.0,0.0,1,0.0,0.0,1,1,0.0,0.0,0.0,university,NaN,NaN
1433,2020-09-26,2020-09-20,UNKNOWN,UNKNOWN,5.0,4.0,0.0,0.0,1.0,46,6.0,0.0,52,49,0.0,0.0,3.0,university,NaN,NaN
2975,2020-10-03,2020-09-27,BUS DRIVER,UNKNOWN,1.0,0.0,0.0,1.0,0.0,1,0.0,0.0,1,0,0.0,1.0,0.0,k12,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111846,2021-03-20,2021-03-14,GROWING DAYS CHILDCARE AND LEARNING CENTER,MIAMI-DADE,0.0,0.0,0.0,0.0,0.0,0,1.0,0.0,1,0,0.0,0.0,1.0,k12,NaN,NaN
112416,2021-03-20,2021-03-14,DIXIE COUNTY LEARNING ACADEMY,DIXIE,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,1,0,0.0,0.0,1.0,k12,NaN,NaN
113586,2021-03-20,2021-03-14,LEE COUNTY JAIL,LEE,0.0,0.0,0.0,0.0,0.0,0,1.0,0.0,1,0,0.0,1.0,0.0,k12,NaN,NaN
113783,2021-03-20,2021-03-14,EASTER SEALS SWFL SECONDARY ACADEMY,MANATEE,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,2,2,0.0,0.0,0.0,k12,NaN,NaN


In [23]:
# fl_dfx[['school','county','school_new','county_new']].drop_duplicates().sort_values(['school','school_new','county']).to_csv('../data/fl_mar28.csv',index=False)

In [13]:
fl_dfx = fl_df.merge(tmp_fl, how='left', on=['school','county'])
fl_dfx['county'] = fl_dfx['county_new']
fl_dfx['school'] = fl_dfx['school_new']
fl_dfx = fl_dfx[['date', 'start_date', 'school', 'county', 'new_cases',
       'new_cases_students', 'new_cases_teachers', 'new_cases_staff',
       'new_cases_unknown', 'symptoms_yes', 'symptoms_no', 'symptoms_idk',
       'cumulative_cases', 'cumulative_cases_students',
       'cumulative_cases_teachers', 'cumulative_cases_staff',
       'cumulative_cases_unknown', 'type']].copy()


gb_cols = ['new_cases', 'new_cases_students', 'new_cases_teachers', 'new_cases_staff',
 'new_cases_unknown', 'symptoms_yes', 'symptoms_no', 'symptoms_idk',
 'cumulative_cases', 'cumulative_cases_students', 'cumulative_cases_teachers',
 'cumulative_cases_staff', 'cumulative_cases_unknown']

fl_dfx['start_date'] = fl_dfx['start_date'].astype(str)
fl_dfx['date'] = fl_dfx['date'].astype(str)
fl_dfx['school'] = fl_dfx['school'].astype(str)
fl_dfx['county'] = fl_dfx['county'].astype(str)
fl_dfx['type'] = fl_dfx['type'].astype(str)

fl_dfx[gb_cols] = fl_dfx[gb_cols].replace(',','', regex=True)

for i in gb_cols:
    fl_dfx[i] = np.nan_to_num(fl_dfx[i].astype(float))
    
fl_dfx = fl_dfx.groupby(['school','county','type','date','start_date'
                ]).agg({i:np.nansum for i in gb_cols}).reset_index()

In [ ]:
fl_dfg = fl_dfx.groupby(['start_date']).agg({'new_cases':'sum',
                                               'new_cases_students':'sum',
                                               'new_cases_teachers':'sum',
                                               'new_cases_staff':'sum',
                                               'new_cases_unknown':'sum',
                                               'symptoms_yes':'sum',
                                               'symptoms_no':'sum',
                                               'symptoms_idk':'sum',
                                               'cumulative_cases':'sum',
                                               'cumulative_cases_students':'sum',
                                               'cumulative_cases_teachers':'sum',
                                               'cumulative_cases_staff':'sum',
                                               'cumulative_cases_unknown':'sum',
                                               }).reset_index()

fl_dfg['new_cases_employee'] = fl_dfg[['new_cases_teachers',
                                       'new_cases_staff']].sum(axis=1)

In [95]:
# fl_dfg = fl_dfx.loc[fl_dfx['type']=='k12'].groupby(['start_date'
#                                        ]).agg({'new_cases':'sum',
#                                                'new_cases_students':'sum',
#                                                'new_cases_teachers':'sum',
#                                                'new_cases_staff':'sum',
#                                                'new_cases_unknown':'sum',
#                                                'symptoms_yes':'sum',
#                                                'symptoms_no':'sum',
#                                                'symptoms_idk':'sum',
#                                                'cumulative_cases':'sum',
#                                                'cumulative_cases_students':'sum',
#                                                'cumulative_cases_teachers':'sum',
#                                                'cumulative_cases_staff':'sum',
#                                                'cumulative_cases_unknown':'sum',
#                                                }).reset_index()

# fl_dfg['new_cases_employee'] = fl_dfg[['new_cases_teachers',
#                                        'new_cases_staff']].sum(axis=1)

In [14]:
fl_dfx['new_cases_employee'] = fl_dfx[['new_cases_teachers',
                                       'new_cases_staff']].sum(axis=1)

In [107]:
# fl_dfx.to_csv('../data/florida_schools.csv',index=False)

In [18]:
# fl_dfx.loc[fl_dfx['type']!='k12'].to_csv('../data/fl_univs.csv',index=False)

In [19]:
# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

creds = ServiceAccountCredentials.from_json_keyfile_name('../client_secret.json', scope)
client = gspread.authorize(creds)

fla_data = client.open('COVID-data_Florida-univ')
fl_df = pd.DataFrame(fla_data.worksheet('univ').get_all_records(default_blank=np.nan))

In [22]:
gb_cols = ['new_cases',
       'new_cases_students', 'new_cases_teachers', 'new_cases_staff',
       'new_cases_unknown', 'symptoms_yes', 'symptoms_no', 'symptoms_idk',
       'cumulative_cases', 'cumulative_cases_students',
       'cumulative_cases_teachers', 'cumulative_cases_staff',
       'cumulative_cases_unknown', 'new_cases_employee']

In [27]:
# fl_df.groupby(['school','county','start_date']).agg({i:np.nansum for i in gb_cols}).reset_index().to_csv('../data/fl_univ.csv',index=False)

In [57]:
fl_df.loc[(fl_df['school'].str.contains(' CAMPU'))&\
          (~(fl_df['school'].str.contains(' CAMPUS'))),
         'school'] = fl_df.loc[(fl_df['school'].str.contains(' CAMPU'))&\
                               (~(fl_df['school'].str.contains(' CAMPUS'))),
                               'school'].str.replace(' CAMPU',' CAMPUS')
fl_df.loc[(fl_df['school'].str.contains(' SCHOO'))&\
          (~(fl_df['school'].str.contains(' SCHOOL'))),
         'school'] = fl_df.loc[(fl_df['school'].str.contains(' SCHOO'))&\
                               (~(fl_df['school'].str.contains(' SCHOOL'))),
                               'school'].str.replace(' SCHOO',' SCHOOL')
fl_df.loc[(fl_df['school'].str.contains(' SCHO'))&\
          (~(fl_df['school'].str.contains(' SCHOOL'))),
         'school'] = fl_df.loc[(fl_df['school'].str.contains(' SCHO'))&\
                               (~(fl_df['school'].str.contains(' SCHOOL'))),
                               'school'].str.replace(' SCHO',' SCHOOL')
fl_df.loc[(fl_df['school'].str.contains(' SCH'))&\
          (~(fl_df['school'].str.contains(' SCHOOL'))),
         'school'] = fl_df.loc[(fl_df['school'].str.contains(' SCH'))&\
                               (~(fl_df['school'].str.contains(' SCHOOL'))),
                               'school'].str.replace(' SCH',' SCHOOL')
fl_df['school'] = fl_df['school'].str.replace('ELEMENTART','ELEMENTARY')
fl_df['school'] = fl_df['school'].str.replace('ELEMENTRY','ELEMENTARY')
fl_df.loc[(fl_df['school'].str.contains(' ELEMEN'))&\
          (~(fl_df['school'].str.contains(' ELEMENTARY')))&\
          (~(fl_df['school'].str.contains('ELEM/'))),
          'school'] = fl_df.loc[(fl_df['school'].str.contains(' ELEMEN'))&\
                               (~(fl_df['school'].str.contains(' ELEMENTARY')))&\
                               (~(fl_df['school'].str.contains('ELEM/'))),'school'].str.replace('ELEMEN','ELEMENTARY')
fl_df.loc[(fl_df['school'].str.contains(' ELEME'))&\
          (~(fl_df['school'].str.contains(' ELEMENTARY')))&\
          (~(fl_df['school'].str.contains('ELEM/'))),
          'school'] = fl_df.loc[(fl_df['school'].str.contains(' ELEME'))&\
                               (~(fl_df['school'].str.contains(' ELEMENTARY')))&\
                               (~(fl_df['school'].str.contains('ELEM/'))),'school'].str.replace('ELEME','ELEMENTARY')
fl_df.loc[(fl_df['school'].str.contains(' ELEM'))&\
          (~(fl_df['school'].str.contains(' ELEMENTARY')))&\
          (~(fl_df['school'].str.contains('ELEM/'))),
          'school'] = fl_df.loc[(fl_df['school'].str.contains(' ELEM'))&\
                               (~(fl_df['school'].str.contains(' ELEMENTARY')))&\
                               (~(fl_df['school'].str.contains('ELEM/'))),'school'].str.replace('ELEM','ELEMENTARY')
fl_df.loc[(fl_df['school'].str.contains(' ACAD'))&\
          (~(fl_df['school'].str.contains(' ACADEMY')))&\
          (~(fl_df['school'].str.contains(' ACADEMIES')))&\
          (~(fl_df['school'].str.contains(' ACADEMI'))),
          'school'] = fl_df.loc[(fl_df['school'].str.contains(' ACAD'))&\
                          (~(fl_df['school'].str.contains(' ACADEMY')))&\
                          (~(fl_df['school'].str.contains(' ACADEMIES')))&\
                          (~(fl_df['school'].str.contains(' ACADEMI'))),
                          'school'].str.replace(' ACAD',' ACADEMY')
fl_df['school'] = fl_df['school'].str.replace('ACADEMYAEMY','ACADEMY')

fl_df['school'] = fl_df['school'].str.replace('HIGHSCHOOL','HIGH')

del_words = ['REFUSED','42 NORTH','N/A','NA','UNKNOWN',
             'BUS DRIVER','CYPRESS SUITS B','AVEDA INSTITUTE',
            'ASBURY THEOLOGICAL SEMINARY SCHOOL',
            'NY INSTITUTE OF ART & DESIGN','PRIMROSE ACADEMY',
             'SANTA FE UNIVERSITY ONLINE','UNDISCLOSED',
             'USF CUSTODIAL WORKER','CHILD DEVELOPMENT CENTER',
            'BUIQUI AEROSPACE CORP','ALCO LEARNING CENTER','ADULT EDUCATION CENTER',
            'CAMPBELL UNIVERSITY','PRE-SCHOOL TEACHER','NASA','BUIQUI AEROSPACE CORP',
             'OSCELOA','KAISER UNIVERSITY','MARIANO MORENO CULINARY INSTITUTE',
            'ST LOUIS CATHOLIC SCHOOL','ST CHRISTOPHER MONTESSORI SCHOOL',
            'THE LEARNING NEST','K-8 LEENWOOD','PALM BEACH SCHOOL DISTRICT',
             'BROWARD INSTITUTE OF PUBLIC SAFETY','PHOENIX UNIVERSITY',
             'SPACE OF MIND DEL RAY BEACH','AUBURN UNIVERSITY',
            'UNIVERSITY OF NORTH CAROLINA AT CHAPEL HILL','UNITED STATES AIR FORCE',
             'THE ARMRY','UNITED STATES AIR FORCE','STANDFORD DORM',
            'SAN CHRISTOPHER BY THE SEA','RASMUSSEN UNIVERSITY','PASCO COUNTY PUBLIC TRANSPORTATION',
            'COMPLIANCE SPECIALIST GRADE 1','LSU','FLORIDA',
            'COMPLIANCE SPECIALIST GRADE 1','WINHOLD']
del_more = ['ALPHA KAI OMEGA - CHAPTER MEETING AT LOCAL CHURCH',
 'DECLINED', 'DECLINED TO GIVE', 'DID NOT PROVIDE', 
 'DRIVER/ CLERK', 'HEALTH CARE DISTRICT FOR PALM BEACH COUNTY',
 'JANITORIAL', 'MIAMI DADE COUNTY MAINTENCE FOR PUBLIC SCHOOLS',
 'NO NAME', 'ONLINE SCHOOLING', 'ORANGE COUNTY PUBLIC SCHOOLS ',
 'PASCO COUNTY SCHOOL DISTRICT', 'PATCHES', 'PPREC',
 'PRIVATE MEDICAL CARE - RENAISSANCE', 'REFUSED INFO',
 'SAME AS ABOVE', 'SAME: CORK ELEMENTARY', 
 'SCHOOL DISTRICT OF MANATEE COUNTY', 'STUDENT NUTRITION',
 'SUBSTITUTE TEACHER', 'TULANE UNIVERSITY', 
 'UNSPECIFIED TRENTON PRE K SCHOOL', '1500 BISCAYNE BLVD', 
 'DADE COUNTY SCHOOL BOARD', 'MCPS BUS DRIVER', 
 'MIAMI DADE COUNTY CENTRAL WEST BUS', 'MIAMI DADE PUBLIC SCHOOL',
 'REDLAND CHRISTIAN MIGRANT ASSOCIATION', 'RILEY',
 'ST JOHNS COUNTY SCHOOL DISTRICT', 'TEACHER', 'WISH NOT TO DISCLOSE',
 'YWCA MIAMI', 'AIRALA EYE INSTITUTE',
 'ASSISTANT TEACHER / LEARNING LINKS SCHOOL', 'FLORIDA HIGH',
 'LIGHTHOUSE GRILL', 'MIAMI', 'WASHINGTON COUNTY SCHOOL DISTRICT',
 'WORKS CLEANING AND REPAIRING', 'MCPS DISTRICT OFFICE',
 'X RAY SCHOOL', 'DEPARTMENT OF EXCEPTIONAL STUDENT EDUCATION',
 'DPT OF FOOD AND NUTRITION', 
 'EAST COAST MIGRANT HEADSTART PROJECT - LA FAMILIA CEN',
 'SAINT VINCENT RIVERSIDE HOSPITAL/ ORANGE PARK MEDIC',
 'SAINT VINCENT RIVERSIDE HOSPITAL/ ORANGE PARK MEDICA',
 'LAKE ERIE COLLEGE OF MEDICINE',
 'DUVAL COUNTY SCHOOL BOARD', 'MIAMI DADE PUBLIC SCHOOLS - BUS DRIVER',
 'MIAMI-DADE COUNTY PUBLIC SCHOOL', 'POLYTECHNIC UNIVERSITY OF PUERTO RICO',
 'THE DEPARTMENT OF TRANSPORTATION', 'UNIVERSITY OF NORTH WESTERN OHIO',
 '3451 EDGEWOOD ROAD', 'CHILD ACADEMY', 'CHILDREN OF WOODBINE PRESCHOOL',
 'FT MCCOY', 'GAMMA PHI BETA', 
 'GREATER LITTLE ROCK PRESCHOOL BAPTIST CHURCH', 'ISCHS',
 'KINDERGARTEN', 'MCBS BUS AND TRANSPORTATION DEPARTMENT',
 'MCSB TRANSPORTATION AND PARTS', 'MIAMI DADE COUNTY SCHOOLS',
 'PASCO COUNTY SCHOOL', 'SAINT',
 'SCHOOL ADMINISTRTAION - DID NOT WANT TO IDENTIFY', 'ST JOHN BUS DRIVER',
 'BOARD OF COUNTY COMMISSION', 'CURRY COLLEGE', 'LEADERSHIP COURSE',
 'COORDINATOR', 'ALPHA KAI OMEGA-CHAPTER MEETING AT LOCAL CHURCH',
 'COMPLIANCE SPECIALIST GRADE 1']

fl_df = fl_df.loc[~fl_df['school'].isin(del_words)].copy()
fl_df = fl_df.loc[~fl_df['school'].isin(del_more)].copy()

fl_df.loc[fl_df['school']=='BELEN','county'] = 'MIAMI-DADE'
fl_df.loc[fl_df['school']=='BELEN','school'] = 'BELEN JESUIT PREPARATORY SCHOOL'
fl_df.loc[fl_df['school']=='ABC INSTITUTE','county'] = 'MIAMI-DADE'
fl_df.loc[fl_df['school']=='ABC INSTITUTE','school'] = 'ABC ACADEMY'
fl_df.loc[fl_df['school']=='NORTH BAPTIST COLLEGE','school'] = 'NORTH FLORIDA BAPTIST COLLEGE'

fl_df.loc[fl_df['school'].str.contains(','),'school'] = fl_df.loc[fl_df['school'].str.contains(','),'school'].str.replace(',','')

fl_df.loc[fl_df['school'].str.contains('MIDDLE SCHOOL'),
           'school'] = fl_df.loc[fl_df['school'].str.contains('MIDDLE SCHOOL'),
                                  'school'].str.replace('MIDDLE SCHOOL','MIDDLE')

fl_df.loc[fl_df['school'].str.contains('ELEMENTARY SCHOOL'),
       'school'] = fl_df.loc[fl_df['school'].str.contains('ELEMENTARY SCHOOL'),
                          'school'].str.replace('ELEMENTARY SCHOOL','ELEMENTARY')

fl_df.loc[fl_df['school'].str.contains('HIGH SCHOOL'),
           'school'] = fl_df.loc[fl_df['school'].str.contains('HIGH SCHOOL'),
                                  'school'].str.replace('HIGH SCHOOL','HIGH')

fl_df.loc[fl_df['school'].str.contains('SENIOR HIGH'),
           'school'] = fl_df.loc[fl_df['school'].str.contains('SENIOR HIGH'),
                                  'school'].str.replace('SENIOR HIGH','HIGH')
fl_df['school'] = fl_df['school'].str.replace('ENGINEEERING','ENGINEERING')
fl_df['school'] = fl_df['school'].str.replace('WYNNBROOK','WYNNEBROOK')
fl_df['school'] = fl_df['school'].str.replace('INSTITUE','INSTITUTE')
fl_df['school'] = fl_df['school'].str.replace('ELEMENTARYANRTY','ELEMENTARY')
fl_df['school'] = fl_df['school'].str.replace('ELEMENTARYANTRY','ELEMENTARY')
fl_df['school'] = fl_df['school'].str.replace('CHARTYER','CHARTER')

fl_df['school'] = fl_df['school'].str.replace('UNIVESITY','UNIVERSITY')
fl_df['school'] = fl_df['school'].str.replace('FLOEIDA','FLORIDA')

fl_df.loc[(fl_df['county']=='MIAMI-DADE')&\
          (fl_df['school'].str.contains('ACADEMY OF EXCELLENCE')),
          'school'] = "ARTS ACADEMY OF EXCELLENCE"


out_count = ['MIAMI-DADE','HILLSBOROUGH', 'PALM BEACH','ESCAMBIA',
             'DADE', 'DUVAL', "PASCO", "LEON", 'BROWARD', "ORANGE"]
for cc in out_count:
    fl_df.loc[(fl_df['county']=='UNKNOWN')&\
          (fl_df['school'].str.contains(cc+" - ")),
          'county'] = cc
    fl_df.loc[(fl_df['county']=='UNKNOWN')&\
              (fl_df['school'].str.contains(cc)),
              'school'] = fl_df.loc[(fl_df['county']=='UNKNOWN')&\
                                (fl_df['school'].str.contains(cc)),
                                'school'].str.replace(cc+" - ",'')

fl_df.loc[fl_df['school']=='WILDWOOD AT FLORIDA STATE UNIVERSITY','school'] = 'FLORIDA STATE UNIVERSITY'
fl_df.loc[fl_df['school']=='SIGMA PI FRATERNITY AT FLORIDA STATE UNIVERSITY','school'] = 'FLORIDA STATE UNIVERSITY'
fl_df.loc[(fl_df['school'].str.contains('BLOOMINGDALE HIGH')),
          'county'] = "HILLSBOROUGH"
fl_df.loc[(fl_df['school'].str.contains('ALLEN NEASE HIGH')),
          'county'] = "ST. JOHNS"
fl_df.loc[(fl_df['school'].str.contains('ALLEN NEASE HIGH')),
          'school'] = "ALLEN D NEASE HIGH"
fl_df.loc[fl_df['school']=='HENRY REEVES K THRU 8','county'] = 'MIAMI-DADE'
fl_df.loc[fl_df['school']=='HENRY REEVES K THRU 8','school'] = 'HENRY E S REEVES K-8 CENTER'
fl_df.loc[fl_df['school']=='TAYLOR - PERRY PRIMARY SCHOOL (PERRY) (PERRY','county'] = 'TAYLOR'
fl_df.loc[fl_df['school']=='TAYLOR - PERRY PRIMARY SCHOOL (PERRY) (PERRY','school'] = 'PERRY ELEMENTARY'
fl_df.loc[fl_df['school'].str.contains('GWINNETT'),'county'] = 'ORANGE'
fl_df.loc[fl_df['school']=='FIRST BAPTIST ACADEMY OF JACKSONVILLE BATRAM','county'] = 'DUVAL'
fl_df.loc[fl_df['school']=='FIRST BAPTIST ACADEMY OF JACKSONVILLE BATRAM','school'] = 'FIRST BAPTIST ACADEMY OF JACKSONVILLE'
fl_df.loc[fl_df['school']=='FLORIDA ATLANTIC UNIVERSITY','county'] = 'PALM BEACH'
fl_df.loc[fl_df['school']=='FLORIDA STATE UNIVERSITY','county'] = 'LEON'
fl_df.loc[fl_df['school']=='UNIVERSITY OF FLORIDA','county'] = 'ALACHUA'

fl_df.loc[fl_df['school']=='UCF','school'] = 'UNIVERSITY OF CENTRAL FLORIDA'
fl_df.loc[fl_df['school']=='FSU','county'] = 'LEON'
fl_df.loc[fl_df['school']=='PARK ON LAYFETTE','school'] = 'FLORIDA STATE UNIVERSITY'
fl_df.loc[fl_df['school']=='FORUM AT TALLAHASSEE','school'] = 'FLORIDA STATE UNIVERSITY'
fl_df.loc[fl_df['school']=='FLORIDA STATE UNIVERSITY COLLEGE OF LAW','school'] = 'FLORIDA STATE UNIVERSITY'
fl_df.loc[fl_df['school']=='FLORIDA STATE','school'] = 'FLORIDA STATE UNIVERSITY'
fl_df.loc[fl_df['school']=='FIU UNIVERSITY-BUILDING MANAGER','school'] = 'FLORIDA INTERNATIONAL UNIVERSITY'
fl_df.loc[fl_df['school']=='FIU DORMITORY LVN515','school'] = 'FLORIDA INTERNATIONAL UNIVERSITY'
fl_df.loc[fl_df['school']=='NORMAN HALL FLORIDA STATE UNIVERSITY','school'] = 'FLORIDA STATE UNIVERSITY'
fl_df.loc[fl_df['school']=='FSU','school'] = 'FLORIDA STATE UNIVERSITY'
fl_df.loc[fl_df['school']=='GAINSVILLE CITY COLLEGE','county'] = 'ALACHUA'
fl_df.loc[fl_df['school']=='GALAXY E3 ELEMENTARY','county'] = 'PALM BEACH'
fl_df.loc[fl_df['school']=='CORTIVA INSTITUTE','county'] = 'ORANGE'
fl_df.loc[fl_df['school']=='VALENCIA COLLEGE','school'] = 'VALENCIA COLLEGE- WEST CAMPUS'
fl_df.loc[fl_df['school']=='YULEE HALL','school'] = 'UNIVERSITY OF FLORIDA'
fl_df.loc[fl_df['school']=='UNIVERISTY GREENS','school'] = 'FLORIDA STATE UNIVERSITY'
fl_df.loc[fl_df['school']=='DORMAN HALL','school'] = 'FLORIDA STATE UNIVERSITY'
fl_df.loc[fl_df['school']=='FLORIDA INTERNATION UNIVERSITY','school'] = 'FLORIDA INTERNATIONAL UNIVERSITY'
fl_df.loc[fl_df['school']=='FLORIDA INTERNATIONAL UNIVERSITY','county'] = 'MIAMI-DADE'
fl_df.loc[fl_df['school']=='JOHN IVES SMITH ELEMENTARY', 'school'] = 'JOHN I SMITH K-8 CENTER'
fl_df.loc[fl_df['school']=='BETH JACOB HIGH', 'county'] = 'MIAMI-DADE'
fl_df.loc[fl_df['school']=='BETH JACOB HIGH', 'school'] = 'BETH JACOB HIGH INC'
fl_df.loc[fl_df['school']=='EASTERN FLORIDA STATE COLLEGE', 'county'] = 'BREVARD'
fl_df.loc[fl_df['school']=='EASTERN FLORIDA STATE COLLEGE', 'school'] = 'EASTERN FLORIDA STATE COLLEGE- COCOA CAMPUS'
fl_df.loc[fl_df['school']=='WESTVIEW PRIMARY SCHOOL', 'county'] = 'DUVAL'
fl_df.loc[fl_df['school']=='WESTVIEW PRIMARY SCHOOL', 'school'] = 'WESTVIEW K-8'
fl_df.loc[fl_df['school']=='UNIVERSITY OF MIAMI/KEY BISCAYNE', 'county'] = 'MIAMI-DADE'
fl_df.loc[fl_df['school']=='UNIVERSITY OF MIAMI/KEY BISCAYNE', 'school'] = 'UNIVERSITY OF MIAMI'
fl_df.loc[fl_df['school']=='EAST LAKE ELEMENTARY', 'county'] = 'OSCEOLA'
fl_df.loc[fl_df['school']=='ST. BRENDAN ELEMENTARY','county'] = 'MIAMI-DADE'
fl_df.loc[fl_df['school']=='ST. BRENDAN ELEMENTARY','school'] = 'SAINT BRENDAN ELEMENTARY'#&\
fl_df.loc[fl_df['school'].str.contains('GAINESVILLE CITY COLLEGE'),'county'] = 'ALACHUA'
fl_df.loc[fl_df['school']=='TAMPA HEIGHTS', 'county'] = 'HILLSBOROUGH'
fl_df.loc[fl_df['school']=='TAMPA HEIGHTS', 'school'] = 'TAMPA HEIGHTS ELEMENTARY MAGNET'
fl_df.loc[fl_df['school']=='HILLSBOROUGH COMMUNITY COLLEGE', 'county'] = 'HILLSBOROUGH'
fl_df.loc[fl_df['school']=='HILLSBOROUGH COMMUNITY COLLEGE', 'school'] = 'HILLSBOROUGH COMMUNITY COLLEGE- DALE MABRY CAMPUS'
fl_df.loc[fl_df['school'].str.contains('MIAMI-DADE COLLEGE'),'county'] = 'MIAMI-DADE'
fl_df.loc[fl_df['school'].str.contains('MIAMI-DADE COLLEGE'),'school'] = 'MIAMI DADE COLLEGE'
fl_df.loc[fl_df['school']=='MATER ACADEMY BAY ELEMENTARY', 'county'] = 'MIAMI-DADE'
fl_df.loc[fl_df['school']=='MATER ACADEMY ELEMENTARY', 'school'] = 'MATER ACADEMY BAY ELEMENTARY'
fl_df.loc[fl_df['school']=='VARSITY LAKE MIDDLE', 'school'] = 'VARSITY LAKES MIDDLE'
fl_df.loc[(fl_df['school']=='RIVERVIEW HIGH')&(fl_df['county']=='UNKNOWN'), 'county'] = 'SARASOTA'
fl_df.loc[fl_df['school']=='RENNIANCE CHARTER','school'] = 'RENAISSANCE CHARTER'
fl_df.loc[fl_df['school']=='HIS HOUSE ACADEMY', 'county'] = 'MIAMI-DADE'
fl_df.loc[fl_df['school']=='HIS HOUSE ACADEMY', 'school'] = 'HIS HOUSE'
fl_df.loc[fl_df['school']=='SOUTHSHORE MONTESSORI SCHOOL', 'school'] = 'SOUTHSHORE CHARTER ACADEMY'
fl_df.loc[fl_df['school']=='ARCHIMEDEAN SCHOOL', 'county'] = 'MIAMI-DADE'
fl_df.loc[fl_df['school']=='ARCHIMEDEAN SCHOOL', 'school'] = 'ARCHIMEDEAN ACADEMY'
fl_df.loc[fl_df['school']=='DIVINE SAVIOR ACADEMY', 'school'] = 'DIVINE SAVIOR LUTHERAN ACADEMY'
fl_df.loc[fl_df['school']=='RIVIERA PREPARATORY SCHOOL', 'county'] = 'PALM BEACH'
fl_df.loc[fl_df['school']=='RIVIERA PREPARATORY SCHOOL', 'school'] = 'RIVIERA BEACH PREPARATORY & ACHIEVEMENT ACADEMY'
fl_df.loc[fl_df['school']=='NORTH LAKE ELEMENTARY', 'county'] = 'POLK'
fl_df.loc[fl_df['school']=='NORTH LAKE ELEMENTARY', 'school'] = 'NORTH LAKELAND ELEMENTARY OF CHOICE'
fl_df.loc[fl_df['school']=='SOUTHERN TECHNICAL COLLEGE', 'county'] = 'ORANGE'
fl_df.loc[fl_df['school']=='NORTH FLORIDA BAPTIST COLLEGE', 'county'] = 'JACKSON'
fl_df.loc[fl_df['school']=='THE FLORIDA SCHOOL FOR THE DEAF AND BLIND', 'county'] = 'ST. JOHNS'
fl_df.loc[fl_df['school']=='FLIGHT SAFETY AVIATION ORLANDO', 'county'] = 'ORANGE'
fl_df.loc[fl_df['school']=="CHATMAN'S DAYCARE INC",'school'] = "CHATMAN'S EARLY LEARNING CHRISTIAN ACADEMY"
fl_df.loc[fl_df['school']=="PARKWAY MIDDLE",'county'] = "BROWARD"
fl_df.loc[fl_df['school']=='BROWN ACADEMY','county'] = "LEE"
fl_df.loc[fl_df['school']=='BRIDGEPREP ACADEMY OF INTERAMERICAN','county'] = "MIAMI-DADE"
fl_df.loc[fl_df['school']=='ADF AIRWAYS FLIGHT SCHOOL','school'] = "ADF AIRWAYS MIAMI FLIGHT SCHOOL"
fl_df.loc[fl_df['school']=='ADF AITWAY','school'] = "ADF AIRWAYS MIAMI FLIGHT SCHOOL"
fl_df.loc[fl_df['school']=='ADS AIRWAYS','school'] = "ADF AIRWAYS MIAMI FLIGHT SCHOOL"
fl_df.loc[fl_df['school']=='FLORIDA TECHNICAL COLLEGE','county'] = "VOLUSIA"
fl_df.loc[fl_df['school']=='ALONZO AND TRACY MOURNING','school'] = "ALONZO & TRACY MOURNING HIGH"
fl_df.loc[fl_df['school']=='AVENTURA CITY OF EXCELLENT','school'] = "AVENTURA CITY OF EXCELLENCE SCHOOL"
fl_df.loc[fl_df['school']=='BISCAYNE GARDEN ELEMENTARY','school'] = "BISCAYNE GARDENS ELEMENTARY"
fl_df.loc[fl_df['school']=='ULTIMATE MEDICAL ACADEMY','county'] = "HILLSBOROUGH"
fl_df.loc[fl_df['school']=='SUMMIT QUESTA','school'] = "SUMMIT-QUESTA MONTESSORI SCHOOL"
fl_df.loc[fl_df['school']=='NATIONAL AVIATION ACADEMY','county'] = "HILLSBOROUGH"
fl_df.loc[fl_df['school']=='KATZ HILLEL DAY SCHOOL','school'] = "KATZ HILLEL DAY SCHOOL OF BOCA RATON"
fl_df.loc[fl_df['school']=='HARTSFIELD ELEMENTARY','school'] = "FRANK HARTSFIELD ELEMENTARY"
fl_df.loc[fl_df['school']=='GIFFORD YOUTH ACHIVEMENT CENTER','school'] = 'GIFFORD YOUTH ACHIEVEMENT CENTER'
fl_df.loc[fl_df['school']=='TAYLOR - PERRY PRIMARY SCHOOL','school'] = 'PERRY ELEMENTARY'
fl_df.loc[fl_df['school']=='EMBRY-RIDDLE AERONAUTICAL UNIVERSITY-DAYTONA BEACH','school'] = 'EMBRY-RIDDLE AERONAUTICAL UNIVERSITY - DAYTONA BEACH'
fl_df.loc[fl_df['school']=='AMBRIY RIDDLE AERONATUTICAL UNIVERSITY','school'] = 'EMBRY-RIDDLE AERONAUTICAL UNIVERSITY - DAYTONA BEACH'
fl_df.loc[fl_df['school']=='EMBRY-RIDDLE AERONAUTICAL UNIVERSITY - DAYTONA BEAC','school'] = 'EMBRY-RIDDLE AERONAUTICAL UNIVERSITY - DAYTONA BEACH'
fl_df.loc[fl_df['school']=='OUR LADY OF THE LAKES','school'] = 'OUR LADY OF THE LAKES CATHOLIC SCHOOL'
fl_df.loc[fl_df['school']=='ST THERESA CATHOLIC SCHOOL','school'] = 'SAINT THERESA CATHOLIC SCHOOL'

fl_df = fl_df.loc[~fl_df['school'].str.contains('MAINTENANC')].copy()

fl_df['school'] = fl_df['school'].str.replace('.','. ')
fl_df['school'] = fl_df['school'].str.replace('.','')
fl_df['school'] = fl_df['school'].str.replace('  ',' ')
fl_df = fl_df.loc[~(fl_df['school'].str.contains('BUIQUI AEROSPACE CORP'))].copy()
fl_df.loc[fl_df['school']=='UNIVERSITY OF ST AUGUSTINE','school'] = 'UNIVERSITY OF ST AUGUSTINE FOR HEALTH SCIENCES'
fl_df.loc[fl_df['school'].str.contains('UNIVERSITY OF ST AUGUSTINE FOR HEALTH SCIENCES-ST'),'school'] = 'UNIVERSITY OF ST AUGUSTINE FOR HEALTH SCIENCES - ST'
fl_df.loc[fl_df['school'].str.contains('UNIVERSITY OF ST AUGUSTINE FOR HEALTH SCIENCES - ST'),'school'] = 'UNIVERSITY OF ST AUGUSTINE FOR HEALTH SCIENCES - ST AUGUSTINE'
fl_df.loc[fl_df['school'].str.contains('UNIVERSITY OF ST AUGUSTINE FOR HEALTH SCIENCES-M'),'school'] = 'UNIVERSITY OF ST AUGUSTINE FOR HEALTH SCIENCES - M'
fl_df.loc[fl_df['school'].str.contains('UNIVERSITY OF ST AUGUSTINE FOR HEALTH SCIENCES - M'),'school'] = 'UNIVERSITY OF ST AUGUSTINE FOR HEALTH SCIENCES - MIAMI'
fl_df.loc[fl_df['school']=='UNIVERSITY OF ST AUGUSTINE FOR HEALTH SCIENCES','school'] = 'UNIVERSITY OF ST AUGUSTINE FOR HEALTH SCIENCES - ST AUGUSTINE'
fl_df.loc[fl_df['school']=='UNIVERSITY OF ST AUGUSTINE FOR HEALTH SCIENCES - ST AUGUSTINE','county'] = 'ST.JOHNS'
fl_df.loc[fl_df['school'].str.contains('UNIVERSITY OF MIAMI'),'school'] = 'UNIVERSITY OF MIAMI'
fl_df.loc[fl_df['school']=='PRIDE ACADEMY','county'] = 'MIAMI-DADE'
fl_df.loc[fl_df['school']=='PRIDE ACADEMY','school'] = 'OUR PRIDE ACADEMY INC'

tmp2 = fl_df.loc[fl_df['county']=='UNKNOWN'].groupby(['school',
                                               'county']).agg(
            {'date':'count'}).reset_index().sort_values(['date','school'])

for sch in tmp2['school'].values:
    uco = fl_df.loc[(fl_df['school']==sch)&\
                    (fl_df['county']!='UNKNOWN')]['county'].unique()
    if len(uco) == 1:
        fl_df.loc[(fl_df['school']==sch)&\
                  (fl_df['county']=='UNKNOWN'),'county'] = uco[0]

fl_df.loc[(fl_df['school'].str.contains('- '))&\
          (~fl_df['school'].str.contains(' - ')),
          'school'] = fl_df.loc[(fl_df['school'].str.contains('- '))&\
                                (~fl_df['school'].str.contains(' - ')),
                                'school'].str.replace('- ',' - ')

In [58]:
tmp2 = fl_df.loc[fl_df['county']=='UNKNOWN'].groupby(['school','county']).agg(
            {'date':'count'}).reset_index().sort_values(['date','school'])

In [61]:
tmp2.tail(50)

,school,county,date
181,SAINT PETERSBURG COLLEGE,UNKNOWN,8
193,SOMERSET CITY ARTS CONSERVATORY,UNKNOWN,8
206,ST PAUL METHODIST SCHOOL,UNKNOWN,8
3,ACADEMY SILVER PALMS,UNKNOWN,9
4,ALCO POINCIANA,UNKNOWN,9
14,BENGAMLA,UNKNOWN,9
31,CONCORDIA UNIVERSITY,UNKNOWN,9
38,CRYSTAL ACADEMY PRIVATE SCHOOL,UNKNOWN,9
70,GALEN COLLEGE OF NURSING,UNKNOWN,9
90,IMATTER ACADEMY,UNKNOWN,9


In [47]:
# fl_df.loc[fl_df['school'].str.contains('ST PAUL CHR')]#['school'].unique()

In [62]:
fl_df.loc[fl_df['school'].str.contains('ELEMENTARYANTRY')]['school'].unique()

array([], dtype=object)

In [49]:
# sch_df.loc[(sch_df['school'].str.contains(' & '))]#&\

In [63]:
tmp = fl_df.merge(sch_df[['school','county','school_id']],
                  how='left', on=['school','county'])
tmp.loc[~tmp['school_id'].isna()]['school_id'].nunique()

1307

In [64]:
import difflib 

In [ ]:
       'FIU UNIVERSITY - BUILDING MANAGER',
       'FLORIDA ATLANTIC UNIVERSITY - IT DEPARTMENT',
'RENAISSANCE CHARTER SCHOOL', 'RENNAISANANCE MIDDLE',
       'RENNISANCE CHARTER SCHOOL', 
    'THE ONE AT UNIVERSITY CITY/FLORIDA INTERNATIONAL UNIV','UCF MAIN',
    'USF BOOOKSTORE','FLORIDA STATE UNIV','FLORIDA STATE UNIVERSITY TALLAHASEE',
    'USF','UNIVERSITY OF SOUTHERN FLORIDA', 'UNIVERSITY SOUTH FLORIDA',

In [69]:
for i in tmp2['school'].unique():
    diff_i = difflib.get_close_matches(i,sch_df['school'].values, n=2)
    print([i]+diff_i)

['AVANT GARDE ACADEMY BROWARD', 'AVANT GARDE ACADEMY OF BROWARD', 'AVANT GARDE ACADEMY K-5 BROWARD']
['BEN HILL MIDDLE', 'HILL MIDDLE', 'WINDY HILL MIDDLE']
['CANAL POINT ELEMENTARY', 'SABAL POINT ELEMENTARY', 'NAVY POINT ELEMENTARY']
['CARRIE BRAZER CENTER FOR AUTISM', 'THE HOPE CHARTER CENTER FOR AUTISM', 'CYPRESS LAKE CENTER FOR THE ARTS']
['CENTER ACADEMY', 'CENTRAL ACADEMY', 'TIGER ACADEMY']
['CHARLES DREW MIDDLE', 'CHARLES R DREW MIDDLE', 'CHARLES S RUSHE MIDDLE']
['CHILDREN PARADISE', 'CHILDREN FIRST', "THE CHILDREN'S READING CENTER"]
['CITRUS ELEMENTARY SCHOOL', 'CITRUS ELEMENTARY', 'CITRUS ELEMENTARY']
['CONCHITA ESPINOZA ACADEMY', 'SARASOTA SUNCOAST ACADEMY', 'NEW CHOICES ACADEMY']
['CONLEY']
['CRISDA CHRISTIAN ACADEMY', 'BENNETT CHRISTIANSEN ACADEMY', 'RIVIERA BEACH MARITIME ACADEMY']
['DEER PARK ELM ', 'DEER PARK ELEMENTARY', 'DEER PARK ELEMENTARY']
['DON GIUNTA MIDDLEOL', 'GIUNTA MIDDLE', 'OMNI MIDDLE']
['DORAL ACADEMY PREP', 'DORAL ACADEMY', 'DORAL ACADEMY CHARTER HIGH']


In [ ]:
contact_df = pd.DataFrame(contact_data.get_all_records(default_blank=np.nan))
contact_df = contact_df.loc[~contact_df['setting'].isin(['College/University',
        'Festival/Fair','Camp','Primary/Secondary School','Vocational School'])].copy()
contact_df = contact_df[['date', 'setting', 'raw_new_clusters', 'raw_new_cases',
                         'raw_total_clusters', 'raw_total_cases']]

contact_df.loc[contact_df['raw_new_cases']<0,'raw_new_cases'] = 0
contact_df.loc[contact_df['raw_new_clusters']<0,'raw_new_clusters'] = 0

In [ ]:
lw = 3
fig, ax = plt.subplots(1,2,figsize=(11,4.5),dpi=200)

labs_tx = ['students','employees','statewide']
cols = ['steelblue','darksalmon','.2']
for ci, col in enumerate(['new_cases_students','new_cases_employee','new_cases_state']):
    xvals_i = [date_dict[i] for i in la_dfg['date'].values]
    yvals_i = la_dfg[col].rolling(window=2).mean().values
    ax[0].plot(xvals_i, yvals_i/np.nansum(yvals_i), label=labs_tx[ci], lw=lw, color=cols[ci],
            path_effects=[path_effects.Stroke(linewidth=lw*1.5, foreground='w'),
                          path_effects.Normal()])
    if ci < 2:
        yvals_i = la_dfg[col].rolling(window=2).mean().values
        ax[1].plot(xvals_i, yvals_i, label=labs_tx[ci], lw=lw, color=cols[ci],
                path_effects=[path_effects.Stroke(linewidth=lw*1.5, foreground='w'),
                              path_effects.Normal()])



dates = sorted(la_dfg['date'].values)[1::4]
ax[0].yaxis.set_major_formatter(mtick.PercentFormatter(1.0,decimals=0))
tlabs = ['Percent of total cases '+\
              '(%s %s – %s %s)'%(month_dict[la_dfg['date'].min()[5:7]],
                                 la_dfg['date'].min()[8:],
                                 month_dict[la_dfg['date'].max()[5:7]],
                                 la_dfg['date'].max()[8:]),
        'Weekly new cases reported '+\
              '(%s %s – %s %s)'%(month_dict[la_dfg['date'].min()[5:7]],
                                 la_dfg['date'].min()[8:],
                                 month_dict[la_dfg['date'].max()[5:7]],
                                 la_dfg['date'].max()[8:]),]

for ai,a in enumerate(fig.axes):
    a.set_xticks([date_dict[i] for i in dates])
    a.set_xticklabels([month_dict[i[5:7]]+'\n'+i[8:] for i in dates])
    l = a.legend(loc=2, ncol=1, framealpha=0)
    for text in l.get_texts():
        text.set_color('.4')
        text.set_path_effects([path_effects.Stroke(linewidth=2, foreground='w'),
                               path_effects.Normal()])

    a.tick_params(labelcolor='.3', color='.3')
    a.grid(linewidth=1.25,color='.75',alpha=0.25)
    a.set_title('Louisiana K-12 Schools COVID-19 Data\n'+tlabs[ai],
                x=1,color='.3',ha='right')
    a.set_xlim(date_dict[la_dfg['date'].min()]+5,
                date_dict[la_dfg['date'].max()]+2)

ax[1].text(0.99,-0.15,'source: Louisiana Department of Health\n'+\
        'https://ldh.la.gov/COVID-19Outbreaks',
        color='.5',fontsize='small', va='top', ha='right', transform=ax[1].transAxes)


# plt.savefig('../figs/pngs/louisiana_k12_studentstaff.png',dpi=425,bbox_inches='tight')
# plt.savefig('../figs/pdfs/louisiana_k12_studentstaff.pdf',bbox_inches='tight')
plt.show()

In [ ]:
wid = 4
barw = 4.5
w = 3.2; h = 2.3
ncols = 5; nrows = 4
tups = list(it.product(range(nrows), range(ncols)))
fig, ax = plt.subplots(nrows, ncols, figsize=(ncols*w,nrows*h), dpi=100, sharex=True)#, sharey=True)
plt.subplots_adjust(wspace=0.3,hspace=0.15)
